# **Instruction Fine-tuning** of Alpaca 🦙 on Arabic
link to **Alpaca Native** Repository -> https://github.com/tatsu-lab/stanford_alpaca

In [115]:
!pip install torch==2.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [113]:
#Preparing the environement
!pip install -q datasets
!pip install -q loralib
!pip install -q sentencepiece
!pip install -q git+https://github.com/huggingface/transformers.git
# transformers from source
!pip install git+https://github.com/huggingface/transformers -q
!pip install -q accelerate
!pip install -q bitsandbytes
!pip install -q git+https://github.com/huggingface/peft.git
!pip install torch=2.0

ERROR: Operation cancelled by user
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/req_command.py", line 241, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/commands/install.

## Importing and Pre-processing the Dataset from 🦙's github repo
> https://github.com/tatsu-lab/stanford_alpaca/blob/main/alpaca_data.json

In [ ]:
!wget https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json

--2023-04-08 19:49:20--  https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22773992 (22M) [text/plain]
Saving to: ‘alpaca_data.json’

alpaca_data.json    100%[===================>]  21.72M   132MB/s    in 0.2s    

2023-04-08 19:49:21 (132 MB/s) - ‘alpaca_data.json’ saved [22773992/22773992]



> Translation task on the dataset using **GoogleTrans** package

In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np

In [ ]:
data = load_dataset("json", data_files="/content/alpaca_data.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-2e69f8f21554e13f/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
!pip install -q googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 3.9 MB/s eta 0:00:00


In [ ]:
from googletrans import Translator

> Little sample output

In [ ]:
translator = Translator()
translations = translator.translate(['the lazy dog'],dest='ko')
for translation in translations:
  print(translation.origin, ' -> ', translation.text)

the lazy dog  ->  게으른 개


In [ ]:
def translate_doc(example):
  input = str(example['input'])
  instruction = str(example['instruction'])
  output = str(example['output'])
  #texts = example['input'] + example['instruction'] + example['output']
  #print("input " + input)
  #print("output " + output)
  #print("instruction " + instruction)
  #return { "input": translator.translate(input, src='en', dest='ko'), "instruction": translator.translate(instruction, src='en', dest='ko'), "output": translator.translate(output, src='en', dest='ko')}
  return {"input":translator.translate(example['input'], dest='Punjabi',).text,
          "instruction":translator.translate(example['instruction'], dest='Punjabi',).text,
          "output":translator.translate(example['output'], dest='Punjabi',).text}

In [ ]:
import json
with open("alpaca_data.json", "r") as read_file:
  with open("translated_data_6.json", "w") as write_file:
    data = json.load(read_file)
    all_translated = []
    i = 0
    for data_item in data[50000:60000]:
      translated = translate_doc(data_item)
      all_translated.append(translated)
      print(i)
      i = i+1
    json.dump(all_translated, write_file, ensure_ascii=False,)
#translate_doc()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
with open("translated_data.json", "r") as read_file:
  new_data = json.load(read_file)
  for data_item in new_data:
    print(data_item)

JSONDecodeError: ignored

In [77]:
# pandas load 6 json files
import pandas as pd
df1 = pd.read_json('translated_data_1.json')
# df2 = pd.read_json('translated_data_2.json')
# df3 = pd.read_json('translated_data_3.json')
# df4 = pd.read_json('translated_data_4.json')
# df5 = pd.read_json('translated_data_5.json')
# df6 = pd.read_json('translated_data_6.json')

#df1 = pd.read_excel("alpaca_data1.xlsx")
#df2 = pd.read_excel("alpaca_data2.xlsx")
#df3 = pd.read_excel("alpaca_data3.xlsx")

# concat 3 xlsx files
# df = pd.concat([df1, df2, df3, df4, df5, df6])
df = pd.concat([df1])

In [78]:
df

,input,instruction,output
0,,ਸਿਹਤਮੰਦ ਰਹਿਣ ਲਈ ਤਿੰਨ ਸੁਝਾਅ ਦਿਓ।,1. ਸੰਤੁਲਿਤ ਖੁਰਾਕ ਖਾਓ ਅਤੇ ਬਹੁਤ ਸਾਰੇ ਫਲ ਅਤੇ ਸਬਜ਼...
1,,ਤਿੰਨ ਪ੍ਰਾਇਮਰੀ ਰੰਗ ਕੀ ਹਨ?,"ਤਿੰਨ ਪ੍ਰਾਇਮਰੀ ਰੰਗ ਲਾਲ, ਨੀਲੇ ਅਤੇ ਪੀਲੇ ਹਨ।"
2,,ਪਰਮਾਣੂ ਦੀ ਬਣਤਰ ਦਾ ਵਰਣਨ ਕਰੋ।,"ਇੱਕ ਪਰਮਾਣੂ ਇੱਕ ਨਿਊਕਲੀਅਸ ਦਾ ਬਣਿਆ ਹੁੰਦਾ ਹੈ, ਜਿਸ ..."
3,,ਅਸੀਂ ਹਵਾ ਪ੍ਰਦੂਸ਼ਣ ਨੂੰ ਕਿਵੇਂ ਘਟਾ ਸਕਦੇ ਹਾਂ?,"ਹਵਾ ਪ੍ਰਦੂਸ਼ਣ ਨੂੰ ਘਟਾਉਣ ਦੇ ਕਈ ਤਰੀਕੇ ਹਨ, ਜਿਵੇਂ ਕ..."
4,,ਉਸ ਸਮੇਂ ਦਾ ਵਰਣਨ ਕਰੋ ਜਦੋਂ ਤੁਹਾਨੂੰ ਇੱਕ ਮੁਸ਼ਕਲ ਫੈ...,ਜਦੋਂ ਮੈਂ ਇੱਕ ਉਸਾਰੀ ਕੰਪਨੀ ਵਿੱਚ ਇੱਕ ਪ੍ਰੋਜੈਕਟ ਮੈਨ...
...,...,...,...
9995,,2020 ਅਮਰੀਕੀ ਚੋਣਾਂ ਬਾਰੇ ਇੱਕ ਖਬਰ ਲੇਖ ਦੇ ਮੁੱਖ ਨੁਕ...,2020 ਅਮਰੀਕੀ ਰਾਸ਼ਟਰਪਤੀ ਚੋਣਾਂ ਹਾਲੀਆ ਯਾਦਾਂ ਵਿੱਚ ਸ...
9996,,ਵਧੇਰੇ ਅਕਾਦਮਿਕ ਟੋਨ ਲਈ ਹੇਠਾਂ ਦਿੱਤੇ ਵਾਕ ਨੂੰ ਸੰਪਾਦ...,ਕੰਪਿਊਟਰ ਕਈ ਤਰ੍ਹਾਂ ਦੇ ਕੰਮ ਕਰਨ ਦੀ ਸਮਰੱਥਾ ਦੇ ਕਾਰਨ...
9997,,ਪੰਜ ਆਮ ਚੁਸਤ ਸਾਫਟਵੇਅਰ ਵਿਕਾਸ ਪ੍ਰਕਿਰਿਆਵਾਂ ਦੇ ਨਾਮ ...,ਪੰਜ ਆਮ ਚੁਸਤ ਸਾਫਟਵੇਅਰ ਵਿਕਾਸ ਪ੍ਰਕਿਰਿਆਵਾਂ ਹਨ ਸਕ੍ਰ...
9998,,ਯੂਰੋ ਮੁਦਰਾ ਦਾ ਪੂਰਾ ਨਾਮ ਕੀ ਹੈ?,ਯੂਰੋ ਮੁਦਰਾ ਦਾ ਪੂਰਾ ਨਾਮ ਯੂਰਪੀਅਨ ਸਿੰਗਲ ਕਰੰਸੀ ਹੈ।


In [79]:
import numpy as np

In [80]:
#df.drop(["Unnamed: 0"],axis =1,inplace=True)
df = df.replace(np.nan,'')

In [81]:
df

,input,instruction,output
0,,ਸਿਹਤਮੰਦ ਰਹਿਣ ਲਈ ਤਿੰਨ ਸੁਝਾਅ ਦਿਓ।,1. ਸੰਤੁਲਿਤ ਖੁਰਾਕ ਖਾਓ ਅਤੇ ਬਹੁਤ ਸਾਰੇ ਫਲ ਅਤੇ ਸਬਜ਼...
1,,ਤਿੰਨ ਪ੍ਰਾਇਮਰੀ ਰੰਗ ਕੀ ਹਨ?,"ਤਿੰਨ ਪ੍ਰਾਇਮਰੀ ਰੰਗ ਲਾਲ, ਨੀਲੇ ਅਤੇ ਪੀਲੇ ਹਨ।"
2,,ਪਰਮਾਣੂ ਦੀ ਬਣਤਰ ਦਾ ਵਰਣਨ ਕਰੋ।,"ਇੱਕ ਪਰਮਾਣੂ ਇੱਕ ਨਿਊਕਲੀਅਸ ਦਾ ਬਣਿਆ ਹੁੰਦਾ ਹੈ, ਜਿਸ ..."
3,,ਅਸੀਂ ਹਵਾ ਪ੍ਰਦੂਸ਼ਣ ਨੂੰ ਕਿਵੇਂ ਘਟਾ ਸਕਦੇ ਹਾਂ?,"ਹਵਾ ਪ੍ਰਦੂਸ਼ਣ ਨੂੰ ਘਟਾਉਣ ਦੇ ਕਈ ਤਰੀਕੇ ਹਨ, ਜਿਵੇਂ ਕ..."
4,,ਉਸ ਸਮੇਂ ਦਾ ਵਰਣਨ ਕਰੋ ਜਦੋਂ ਤੁਹਾਨੂੰ ਇੱਕ ਮੁਸ਼ਕਲ ਫੈ...,ਜਦੋਂ ਮੈਂ ਇੱਕ ਉਸਾਰੀ ਕੰਪਨੀ ਵਿੱਚ ਇੱਕ ਪ੍ਰੋਜੈਕਟ ਮੈਨ...
...,...,...,...
9995,,2020 ਅਮਰੀਕੀ ਚੋਣਾਂ ਬਾਰੇ ਇੱਕ ਖਬਰ ਲੇਖ ਦੇ ਮੁੱਖ ਨੁਕ...,2020 ਅਮਰੀਕੀ ਰਾਸ਼ਟਰਪਤੀ ਚੋਣਾਂ ਹਾਲੀਆ ਯਾਦਾਂ ਵਿੱਚ ਸ...
9996,,ਵਧੇਰੇ ਅਕਾਦਮਿਕ ਟੋਨ ਲਈ ਹੇਠਾਂ ਦਿੱਤੇ ਵਾਕ ਨੂੰ ਸੰਪਾਦ...,ਕੰਪਿਊਟਰ ਕਈ ਤਰ੍ਹਾਂ ਦੇ ਕੰਮ ਕਰਨ ਦੀ ਸਮਰੱਥਾ ਦੇ ਕਾਰਨ...
9997,,ਪੰਜ ਆਮ ਚੁਸਤ ਸਾਫਟਵੇਅਰ ਵਿਕਾਸ ਪ੍ਰਕਿਰਿਆਵਾਂ ਦੇ ਨਾਮ ...,ਪੰਜ ਆਮ ਚੁਸਤ ਸਾਫਟਵੇਅਰ ਵਿਕਾਸ ਪ੍ਰਕਿਰਿਆਵਾਂ ਹਨ ਸਕ੍ਰ...
9998,,ਯੂਰੋ ਮੁਦਰਾ ਦਾ ਪੂਰਾ ਨਾਮ ਕੀ ਹੈ?,ਯੂਰੋ ਮੁਦਰਾ ਦਾ ਪੂਰਾ ਨਾਮ ਯੂਰਪੀਅਨ ਸਿੰਗਲ ਕਰੰਸੀ ਹੈ।


In [82]:
from datasets import Dataset, load_dataset
dataset = Dataset.from_pandas(df)

In [83]:
#dataset = dataset.rename_column(" انتاج",'output')
#dataset = dataset.rename_column(" مدخل",'input')
#dataset = dataset.rename_column("تعليمات",'instruction') 

In [84]:
dataset = dataset.remove_columns('__index_level_0__')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/datasets/arrow_dataset.py:563 in wrapper                  │
│                                                                                                  │
│    560 │   │   else:                                                                             │
│    561 │   │   │   self: "Dataset" = kwargs.pop("self")                                          │
│    562 │   │   # apply actual function                                                           │
│ ❱  563 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    564 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    565 │   │   for dataset in datasets:                                                          │
│    566 │   │   │   # Remove task templates if a column mapping of the template is no longer val  │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/datasets/arrow_dataset.py:528 in wrapper                  │
│                                                                                                  │
│    525 │   │   │   "output_all_columns": self._output_all_columns,                               │
│    526 │   │   }                                                                                 │
│    527 │   │   # apply actual function                                                           │
│ ❱  528 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    529 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    530 │   │   # re-apply format to the output                                                   │
│    531 │   │   for dataset in datasets:                                                          │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/datasets/fingerprint.py:511 in wrapper                    │
│                                                                                                  │
│   508 │   │   │                                                                                  │
│   509 │   │   │   # Call actual function                                                         │
│   510 │   │   │                                                                                  │
│ ❱ 511 │   │   │   out = func(dataset, *args, **kwargs)                                           │
│   512 │   │   │                                                                                  │
│   513 │   │   │   # Update fingerprint of in-place transforms + update in-place history of tra   │
│   514                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/datasets/arrow_dataset.py:2064 in remove_columns          │
│                                                                                                  │
│   2061 │   │                                                                                     │
│   2062 │   │   for column_name in column_names:                                                  │
│   2063 │   │   │   if column_name not in dataset._data.column_names:                             │
│ ❱ 2064 │   │   │   │   raise ValueError(                                                         │
│   2065 │   │   │   │   │   f"Column name {column_name} not 

In [85]:
dataset[0]

{'input': '',
 'instruction': 'ਸਿਹਤਮੰਦ ਰਹਿਣ ਲਈ ਤਿੰਨ ਸੁਝਾਅ ਦਿਓ।',
 'output': "1. ਸੰਤੁਲਿਤ ਖੁਰਾਕ ਖਾਓ ਅਤੇ ਬਹੁਤ ਸਾਰੇ ਫਲ ਅਤੇ ਸਬਜ਼ੀਆਂ ਨੂੰ ਸ਼ਾਮਲ ਕਰਨਾ ਯਕੀਨੀ ਬਣਾਓ।\n2. ਆਪਣੇ ਸਰੀਰ ਨੂੰ ਕਿਰਿਆਸ਼ੀਲ ਅਤੇ ਮਜ਼ਬੂਤ \u200b\u200bਰੱਖਣ ਲਈ ਨਿਯਮਿਤ ਤੌਰ 'ਤੇ ਕਸਰਤ ਕਰੋ।\n3. ਕਾਫ਼ੀ ਨੀਂਦ ਲਓ ਅਤੇ ਇਕਸਾਰ ਨੀਂਦ ਦਾ ਸਮਾਂ ਬਣਾਈ ਰੱਖੋ।"}

In [86]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


> Saving a checkpoint to upload the dataset from 🤗

In [87]:
 dataset.push_to_hub("japneets/Alpaca_instruction_fine_tune_Punjabi_small")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

## Fine-tune 🦙 Model using PEFT on instruction fine tuning

> Download the instruction dataset from 🤗 Datasets

In [88]:
from datasets import Dataset, load_dataset
dataset = Dataset.from_pandas(df)

In [89]:
dataset = load_dataset("japneets/Alpaca_instruction_fine_tune_Punjabi_small")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/japneets___parquet/japneets--Alpaca_instruction_fine_tune_Punjabi_small-858bddfde9ee8b72/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [186]:
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model
import bitsandbytes as bnb
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

> Setup 🦙 Pre-trained Model and tokenizer

In [187]:
MICRO_BATCH_SIZE = 2  # this could actually be 5 but i like powers of 2
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 1  # we don't need 3 tbh
LEARNING_RATE = 3e-4  # the Karpathy constant
CUTOFF_LEN = 2048  # 256 accounts for about 96% of the data
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
TRAIN_STEPS=100

In [188]:
LEARNING_RATE

0.0003

In [215]:
model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
tokenizer = LlamaTokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf", add_eos_token=True
)

model = prepare_model_for_int8_training(model)

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
) 
tokenizer.pad_token_id = 0
model = get_peft_model(model, config)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


> Template to be used for the instruction fine tuning 🦙 model

In [190]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 10000
    })
})

In [191]:
len(dataset['train']['input'])

10000

In [192]:
def generate_prompt(data_point):
    # desculpe o desastre de formatação, preciso ser rápido
    if data_point["input"]:
        return f"""ਹੇਠਾਂ ਇੱਕ ਕੰਮ ਦਾ ਵਰਣਨ ਕਰਨ ਵਾਲੀਆਂ ਹਦਾਇਤਾਂ ਹਨ, ਇੱਕ ਐਂਟਰੀ ਦੇ ਨਾਲ ਜੋ ਹੋਰ ਸੰਦਰਭ ਪ੍ਰਦਾਨ ਕਰਦੀ ਹੈ। ਇੱਕ ਜਵਾਬ ਲਿਖੋ ਜੋ ਬੇਨਤੀ ਨੂੰ ਸਹੀ ਢੰਗ ਨਾਲ ਪੂਰਾ ਕਰਦਾ ਹੈ।
### ਨਿਰਦੇਸ਼:
{data_point["instruction"]}
### ਇੰਪੁੱਟ:
{data_point["input"]}
### ਆਉਟਪੁੱਟ:
{data_point["output"]}"""
    else:
        return f"""ਹੇਠਾਂ ਇੱਕ ਕੰਮ ਦਾ ਵਰਣਨ ਕਰਨ ਵਾਲੀਆਂ ਹਦਾਇਤਾਂ ਹਨ। ਇੱਕ ਜਵਾਬ ਲਿਖੋ ਜੋ ਬੇਨਤੀ ਨੂੰ ਸਹੀ ਢੰਗ ਨਾਲ ਪੂਰਾ ਕਰਦਾ ਹੈ
### ਨਿਰਦੇਸ਼:
{data_point["instruction"]}
### ਆਉਟਪੁੱਟ:
{data_point["output"]}"""



def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length"
    )
    # if (result["input_ids"][-1] != tokenizer.eos_token_id
    #   and len(result["input_ids"]) < CUTOFF_LEN
    #   and add_eos_token
    # ):
    #   result["input_ids"].append(tokenizer.eos_token_id)
    #   result["attention_mask"].append(1)
    
    result["labels"] = result["input_ids"].copy()
    return result

In [193]:
def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenize_full_prompt = tokenize(full_prompt)
  #print(full_prompt)
  return tokenize_full_prompt

In [194]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 10000
    })
})

In [211]:
train_val = dataset["train"].train_test_split(
    test_size=2000, shuffle=True, seed=42
)
train_val

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 2000
    })
})

In [212]:
train_data = train_val["train"].map(generate_and_tokenize_prompt)
train_data

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'instruction', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 8000
})

In [213]:
val_data = train_val["test"].map(generate_and_tokenize_prompt)
val_data

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'instruction', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})

In [179]:
#dataset = dataset['train'].shuffle().map(lambda x: tokenize(generate_prompt(x)))

In [180]:
len(dataset['input_ids'])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/datasets/dataset_dict.py:57 in __getitem__                │
│                                                                                                  │
│     54 │                                                                                         │
│     55 │   def __getitem__(self, k) -> Dataset:                                                  │
│     56 │   │   if isinstance(k, (str, NamedSplit)) or len(self) == 0:                            │
│ ❱   57 │   │   │   return super().__getitem__(k)                                                 │
│     58 │   │   else:                                                                             │
│     59 │   │   │   available_suggested_splits = [                                                │
│     60 │   │   │   │   split for split in (Split.TRAIN, Split.TEST, Split.VALIDATION) if split   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'input_ids'

In [ ]:
#dataset['attention_mask'][0]

In [198]:
import transformers

> Training the 🦙 model using instruct fine-tuning

In [216]:
import torch
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        max_steps=TRAIN_STEPS,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=10,
        optim="adamw_torch",
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=50,
        save_steps=50,
        #load_best_model_at_end=True,
        output_dir="lora-alpaca",
        save_total_limit=3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
# old_state_dict = model.state_dict
# model.state_dict=(
#     lambda self, *_, **__: get_peft_model_state_dict(
#         self, old_state_dict()
#     )
# ).__get__(model, type(model))

model = torch.compile(model)


In [217]:
trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss


TrainOutput(global_step=101, training_loss=0.006489307573526212, metrics={'train_runtime': 176.7053, 'train_samples_per_second': 72.437, 'train_steps_per_second': 0.566, 'total_flos': 1.0508168916526694e+18, 'train_loss': 0.006489307573526212, 'epoch': 1.62})

> Let's have an insight of our fine tuned Alpaca using Peft

In [224]:
model

OptimizedModule(
  (_orig_mod): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
          (layers): ModuleList(
            (0-31): 32 x LlamaDecoderLayer(
              (self_attn): LlamaAttention(
                (q_proj): Linear8bitLt(
                  in_features=4096, out_features=4096, bias=False
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=4096, bias=False)
                  )
                )
                (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (v_proj): Linear8bitLt(


In [243]:
model = model.to('cpu')

In [246]:
model.save_pretrained("lora-alpaca_punjabi_small")

In [244]:
torch.cuda.empty_cache()

In [239]:
torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 6            |        cudaMalloc retries: 9         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  36480 MiB |  36662 MiB |   7500 TiB |   7500 TiB |\n|       from large pool |  36257 MiB |  36439 MiB |   7495 TiB |   7495 TiB |\n|       from small pool |    223 MiB |    349 MiB |      5 TiB |      5 TiB |\n|---------------------------------------------------------------------------|\n| Active memory         |  36480 MiB |  36662 MiB |   7500 TiB |   7500 TiB |\n|       from large pool |  36257 MiB |  36439 MiB |

In [245]:
import gc
#del variables
gc.collect()

575

> Push the adapter to🤗

In [247]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [248]:
model.push_to_hub("japneets/Alpaca_instruction_fine_tune_Punjabi_small", use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/japneets/Alpaca_instruction_fine_tune_Punjabi_small/commit/fba7a3db7acc3fa01add9f468b9f1b7fde92b9f2', commit_message='Upload model', commit_description='', oid='fba7a3db7acc3fa01add9f468b9f1b7fde92b9f2', pr_url=None, pr_revision=None, pr_num=None)

## Load you Model from 🤗 and put it to the test

In [1]:
import torch
import bitsandbytes as bnb
from peft import PeftModel, PeftConfig, prepare_model_for_int8_training, LoraConfig, get_peft_model
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

# peft_model_id = "Yasbok/Alpaca_instruction_fine_tune_Arabic"
# config = PeftConfig.from_pretrained(peft_model_id)
peft_model_id = "japneets/Alpaca_instruction_fine_tune_Punjabi_small"

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf",
                                          load_in_8bit=True,
                                          device_map="auto",)
# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [2]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""ਹੇਠਾਂ ਇੱਕ ਕੰਮ ਦਾ ਵਰਣਨ ਕਰਨ ਵਾਲੀਆਂ ਹਦਾਇਤਾਂ ਹਨ, ਇੱਕ ਐਂਟਰੀ ਦੇ ਨਾਲ ਜੋ ਹੋਰ ਸੰਦਰਭ ਪ੍ਰਦਾਨ ਕਰਦੀ ਹੈ। ਇੱਕ ਜਵਾਬ ਲਿਖੋ ਜੋ ਬੇਨਤੀ ਨੂੰ ਸਹੀ ਢੰਗ ਨਾਲ ਪੂਰਾ ਕਰਦਾ ਹੈ।


### ਨਿਰਦੇਸ਼:
{instruction}

### ਇੰਪੁੱਟ:
{input}

### ਆਉਟਪੁੱਟ:"""
    else:
        return f"""ਹੇਠਾਂ ਇੱਕ ਕੰਮ ਦਾ ਵਰਣਨ ਕਰਨ ਵਾਲੀਆਂ ਹਦਾਇਤਾਂ ਹਨ। ਕਿਰਪਾ ਕਰਕੇ ਇੱਕ ਜਵਾਬ ਲਿਖੋ ਜੋ ਬੇਨਤੀ ਨੂੰ ਸਹੀ ਢੰਗ ਨਾਲ ਪੂਰਾ ਕਰਦਾ ਹੈ।



### ਨਿਰਦੇਸ਼:
{instruction}

### ਆਉਟਪੁੱਟ:"""

In [3]:
generation_config = GenerationConfig(
    temperature=0.2,
    top_p=0.75,
    num_beams=4,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("ਆਉਟਪੁੱਟ:", output.split("### ਆਉਟਪੁੱਟ:")[1].strip())

In [4]:
evaluate(input("ਨਿਰਦੇਸ਼: "))

ਨਿਰਦੇਸ਼: ਸਿਹਤਮੰਦ ਰਹਿਣ ਲਈ ਤਿੰਨ ਸੁਝਾਅ ਦਿਓ।
ਆਉਟਪੁੱਟ: ਸਿਹਤਮੰਦ ਰਹਿਣ ਲਈ ਤਿੰਨ ਸੁਝਾਅ ਦਿਓ।

### ਇੰਪੁੱਟ:
ਸਿਹਤਮੰਦ ਰਹਿਣ ਲਈ ਤਿੰਨ ਸੁਝਾਅ ਦਿਓ।
